# Quantum Processor

https://docs.netsquid.org/latest-release/tutorial.quantumprocessor.html

In [1]:
import netsquid.components.instructions as instr
from netsquid.components.qmemory import QuantumMemory

In [2]:
qmemory = QuantumMemory('ExampleQMem', num_positions=1)
instr.INSTR_INIT(qmemory, positions=[0])
instr.INSTR_H(qmemory, positions=[0])
instr.INSTR_MEASURE_X(qmemory, positions=[0])

[0]

In [3]:
from netsquid.qubits import operators as ops
INSTR_XY = instr.IGate('xy_gate', ops.X * ops.Y)

In [4]:
from netsquid.components.qprocessor import QuantumProcessor
import netsquid as ns

qproc = QuantumProcessor('ExampleQPU', num_positions=3, fallback_to_nonphysical=True)
qproc.execute_instruction(instr.INSTR_INIT, [0, 1])
qproc.execute_instruction(instr.INSTR_H, [1])
qproc.execute_instruction(instr.INSTR_CNOT, [1, 0])
m1 = qproc.execute_instruction(instr.INSTR_MEASURE, [0])
m2 = qproc.execute_instruction(instr.INSTR_MEASURE, [1])
m1 == m2
ns.sim_time()

0.0

In [15]:
from netsquid.components.models.qerrormodels import DepolarNoiseModel
from netsquid.components.qprocessor import PhysicalInstruction

phys_instructions = [
    PhysicalInstruction(instr.INSTR_INIT, duration=3),
    PhysicalInstruction(instr.INSTR_H, duration=1, parallel=True, topology=[0, 2]),
    PhysicalInstruction(instr.INSTR_X, duration=1, parallel=True, topology=[0, 2]),
    PhysicalInstruction(instr.INSTR_Z, duration=1, parallel=True, topology=[0, 2]),
    PhysicalInstruction(instr.INSTR_CNOT, duration=4, parallel=True, topology=[(0, 1), (2, 1)]),
    PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=False, quantum_noise_model=DepolarNoiseModel(depolar_rate=0.01, time_independent=True), \
                                             apply_q_noise_after=False, topology=[1]),
    PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=True, topology=[0, 2])
]

noisy_qproc = QuantumProcessor('NoiseQPU', num_positions=3, mem_noise_models=[DepolarNoiseModel(1e7)] * 3, phys_instructions=phys_instructions)

In [48]:
ns.sim_reset()
ns.sim_time()

0.0

In [49]:
noisy_qproc.execute_instruction(instr.INSTR_INIT, [0, 1])
ns.sim_run()
ns.sim_time()

3.0

In [50]:
noisy_qproc.execute_instruction(instr.INSTR_H, [0])
noisy_qproc.execute_instruction(instr.INSTR_CNOT, [0, 1])

ProcessorBusyError: 

## Quantum programs

In [7]:
from netsquid.components.qprogram import QuantumProgram

In [52]:
prog = QuantumProgram(num_qubits=2)
q1, q2 = prog.get_qubit_indices(2)    # get the qubit indices we'll be working with
prog.apply(instr.INSTR_INIT, [q1, q2])
prog.apply(instr.INSTR_H, q1)
prog.apply(instr.INSTR_CNOT, [q1, q2])
prog.apply(instr.INSTR_MEASURE, q1, output_key='m1')
prog.apply(instr.INSTR_MEASURE, q2, otuput_key='m2')

In [53]:
noisy_qproc.reset()
ns.sim_reset()
noisy_qproc.execute_program(prog, qubit_mapping = [2, 1])
ns.sim_run()
ns.sim_time()   # 3 + 1 + 4 + 7 = 15

15.0

In [46]:
class EntangleProgram(QuantumProgram):
    default_num_qubits = 2

    def program(self):
        q1, q2 = self.get_qubit_indices(2)
        self.apply(instr.INSTR_INIT, [q1, q2])
        self.apply(instr.INSTR_H, q1)
        self.apply(instr.INSTR_CNOT, [q1, q2])
        self.apply(instr.INSTR_MEASURE, q1, output_key='m1')
        self.apply(instr.INSTR_MEASURE, q2, output_key='m2')
        yield self.run()

In [54]:
class ControlledQProgram(QuantumProgram):
    defalut_num_qubits = 3

    def program(self):
        q1, q2, q3 = self.get_qubit_indices(3)
        self.apply(instr.INSTR_H, q1)
        self.apply(instr.INSTR_MEASURE, q1, output_key='m1')
        yield self.run()
        # depending on outcome on q1 either flip q2 or q3
        if self.output['m1'][0] == 0:
            self.apply(instr.INSTR_X, q2)
        else:
            self.apply(instr.INSTR_X, q3)
        self.apply(instr.INSTR_MEASURE, q2, output_key='m2')
        self.apply(instr.INSTR_MEASURE, q3, output_key='m3')
        yield self.run(parallel=False)


## Local teleportation example using programs

In [37]:
noisy_qproc.reset()
ns.sim_reset()
ns.set_qstate_formalism(ns.QFormalism.DM)

class TeleportationProgram(QuantumProgram):
    default_num_qubits = 3

    def program(self):
        q0, q1, q2 = self.get_qubit_indices(3)
        # entangle q1 and q2
        self.apply(instr.INSTR_INIT, [q0, q1, q2])
        self.apply(instr.INSTR_H, q2)
        self.apply(instr.INSTR_CNOT, [q2, q1])
        # set q0 to the desired state to be teleported
        self.apply(instr.INSTR_H, q0)
        self.apply(instr.INSTR_S, q0)
        # bell measurement
        self.apply(instr.INSTR_CNOT, [q0, q1])
        self.apply(instr.INSTR_H, q0)
        self.apply(instr.INSTR_MEASURE, q0, output_key='M1')
        self.apply(instr.INSTR_MEASURE, q1, output_key='M2')
        yield self.run()
        # do corrections
        if self.output['M2'][0] == 1:
            self.apply(instr.INSTR_X, q2)
        if self.output['M1'][0] == 1:
            self.apply(instr.INSTR_Z, q2)
        yield self.run()

In [38]:
noisy_qproc.execute_program(TeleportationProgram())
ns.sim_run()
qubit = noisy_qproc.pop(2)
fidelity = ns.qubits.fidelity(qubit, ns.qubits.outerprod((ns.S*ns.H*ns.s0).arr), squared=True)
print(f'{fidelity:.3f}')

0.922
